In [3]:
#import pathlib
from pathlib import Path
from typing import List, Tuple, Union
import csv
#import copy
import numpy as np
#import matplotlib.pyplot as plt
import serpentTools
from serpentTools.parsers.results import ResultsReader
#import click
#import matplotlib.pyplot as plt

serpentTools.settings.rc["serpentVersion"] = "2.1.32"
P = 48 #max no of FA
Z = 6  #max no of slices

AV = 0.6221408E+24

# get atomic wt numbers, store in atomic_wt_dict
with open("isotope_awt_list.csv", mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        atomic_wt_dict = {rows[0]:rows[1] for rows in reader}


def discharge_bu() -> None:
    res_list = []

    with serpentTools.settings.rc:
        serpentTools.settings.rc["verbosity"] = "error"
    dep = serpentTools.read("wh_lfr_dep.m")
    
    return dep

dep = discharge_bu()


def split_name(name: str)->Tuple[int, int]:
    """The functions take a fuel name, separates the P and Z and takes out the indexes
        Note: It s made only for this specific nomeclature, needs modification if you 
        want more fancy names

    Args:
        name (str): Takes in a standard fuel position name "fuelPxxxZxxx"

    Returns:
        Tuple[int, int]: Gives out a tuple with the val of P and Z
    """
    without_fuel = name[4:]
    split_without_fuel  = without_fuel.split("Z")
    p_num = int(split_without_fuel[0][1:])
    z_num = int(split_without_fuel[1])
    return p_num, z_num

### NEW VERSION

#### This is the part to be used inside the run to get the right _dep.m file

set inventory			% - Sets inventory for dep.m file 

Th-227 Th-228 Th-229 Th-230 Th-231 Th-232 Th-233 Th-234
Pa-231 Pa-232 Pa-233
U-232  U-233  U-234  U-235  U-236  U-237  U-238  U-239  U-240  U-241 
Np-235 Np-236 Np-237 Np-238 Np-239
Pu-236 Pu-237 Pu-238 Pu-239 Pu-240 Pu-241 Pu-242 Pu-243 Pu-244
Am-241 Am-242 Am-242m Am-243 Am-244 Am-244m 
Cm-240 Cm-241 Cm-242 Cm-243 Cm-244 Cm-245 Cm-246 Cm-247 Cm-248 Cm-249 Cm-250

Th Pa U Np Pu Am Cm			% - total list

In [4]:
materials = dep.materials.keys()
#create my own dict with fuel vol's
fuel_vol_dict = {}

# Go through each material index
for fuel_vol in materials:
    if fuel_vol == "total":
        # The continue, skips the loop interation at that point
        continue
    # Add the materials until 52 pos, exclude the total ones, big total and lost (maybe there is some way more intuitive)
    # Last column (last BU step in sss2) is taken and summed up for each fuel volume, store in dic wth key mdens_sum
    fuel_vol_dict[fuel_vol] = {"mdens_sum": dep.materials[fuel_vol].mdens[:, -1][0:52].sum(), "serpent_burnup": dep.materials[fuel_vol].burnup[-1]}

for fuel_vol in fuel_vol_dict:
    p_index, z_index = split_name(str(fuel_vol))
    mdens_to_divide_key = f"fuelP1Z{z_index}"
    mdens_to_divide = dep.materials[mdens_to_divide_key].mdens[:, -1][0:52].sum()
    ratio = fuel_vol_dict[fuel_vol]["mdens_sum"] / mdens_to_divide
    fuel_vol_dict[fuel_vol]["mdens_ratio"] = ratio
    #print(ratio)

for fuel_vol in fuel_vol_dict.keys():
    p_index, z_index = split_name(fuel_vol)
    burn_up = fuel_vol_dict[fuel_vol]["serpent_burnup"]
    #print(burn_up)
    if p_index == 1:
        fuel_vol_dict[fuel_vol]["corrected_burnup"] = burn_up
        continue
    
    material_name = f"fuelP{p_index - 1}Z{z_index}"
    ratio = fuel_vol_dict[material_name]["mdens_ratio"]
    corrected_burnup = burn_up * ratio
    fuel_vol_dict[fuel_vol]["corrected_burnup"] = corrected_burnup

average_burnups = {}
for fuel_vol in fuel_vol_dict.keys():
    p_index, _ = split_name(fuel_vol)
    average_burnup = 0
    for zidx in range(1 , Z + 1):
        key_construct = f"fuelP{p_index}Z{zidx}"
        average_burnup += fuel_vol_dict[key_construct]["corrected_burnup"] / Z
    average_burnups[f"P{p_index}"] = average_burnup

average_burnups_axial = {}
for fuel_vol in fuel_vol_dict.keys():
    _, z_index = split_name(fuel_vol)
    average_axial_burnup = 0
    for pidx in range(1 , P + 1):
        key_construct = f"fuelP{pidx}Z{z_index}"
        average_axial_burnup += fuel_vol_dict[key_construct]["corrected_burnup"]
    average_burnups_axial[f"Z{z_index}"] = average_axial_burnup


Executing <Task pending coro=<Kernel.dispatch_queue() running at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/ipykernel/kernelbase.py:510> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f6413ff6ae0>()] created at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/ioloop.py:687] created at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/gen.py:874> took 1.092 seconds


In [5]:
average_burnups_axial

{'Z1': 81.46520405843272,
 'Z2': 216.33793075365776,
 'Z3': 313.28412809224216,
 'Z4': 312.489616482833,
 'Z5': 215.53165120372307,
 'Z6': 81.54178152806843}

In [6]:
sum([value for key, value in average_burnups.items()])


203.44171868649292

In [7]:
sum([value for key, value in average_burnups_axial.items()]) / Z

203.44171868649286

In [8]:
average_burnups

{'P48': 2.3027596118250937,
 'P47': 4.259862717259177,
 'P46': 6.928966187393164,
 'P45': 10.02019538112036,
 'P44': 15.41954841569802,
 'P43': 17.132529143126344,
 'P42': 16.18840089158993,
 'P41': 14.0444368174942,
 'P40': 13.986071547488967,
 'P39': 11.185551746269416,
 'P38': 11.485190872207179,
 'P37': 10.868317957963296,
 'P36': 7.81091188031589,
 'P35': 8.139558709966655,
 'P34': 7.878253902285693,
 'P33': 7.146856331080695,
 'P32': 4.477118140758199,
 'P31': 4.7952410713684035,
 'P30': 4.624577198759219,
 'P29': 3.9853266185183633,
 'P28': 2.2030734981233766,
 'P27': 2.438129605710859,
 'P26': 2.4856868153675884,
 'P25': 2.3254975833280476,
 'P24': 2.019975002893941,
 'P23': 1.6489339496448399,
 'P22': 0.7932933787755411,
 'P21': 0.9996445168443273,
 'P20': 0.9709563690039044,
 'P19': 0.8964983489240965,
 'P18': 0.6399767685942884,
 'P17': 0.4243833306968363,
 'P16': 0.18219838922297904,
 'P15': 0.252038929090344,
 'P14': 0.31578482246287415,
 'P13': 0.3374710299529247,
 'P12':

In [9]:
dep = serpentTools.read("wh_lfr_dep.m")

In [10]:
dir(dep.materials["total"])


Executing <Task pending coro=<Kernel.dispatch_queue() running at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/ipykernel/kernelbase.py:510> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f6415914a20>()] created at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/ioloop.py:687] created at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/gen.py:874> took 0.546 seconds


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_checkCompareObj',
 '_checkTimePoints',
 '_compare',
 '_compareLogPreMsg',
 '_formatLabel',
 '_getColIndices',
 '_getIsoID',
 '_getRowIndices',
 '_slice',
 'activity',
 'addData',
 'adens',
 'burnup',
 'compare',
 'data',
 'days',
 'decayHeat',
 'docAttrs',
 'docEquiv',
 'docParams',
 'get',
 'getValues',
 'ingTox',
 'inhTox',
 'mdens',
 'name',
 'names',
 'photonProdRate',
 'plot',
 'spontaneousFissionRate',
 'toDataFrame',
 'volume',
 'zai']

In [78]:
dayPoints = totalfuel.days
iso = ['plutonium', 'U238']
zai = [94, 922380]

totalfuel = dep.materials["total"]

#totalfuel.data["mass"]

totalfuel.getValues('days', 'mass', totalfuel.days, iso)
#totalfuel.names

array([[ 3076140.,  3221580.],
       [56550500., 56138500.]])

In [74]:
totalfuel.days

array([  0., 280.])

In [61]:
import json

x = dep.materials["fuelP48Z1"].toDataFrame("a", names=[json.dumps(totalfuel.names)], time="days")
x

ValueError: Could not find isotope names ["Th227", "Th228", "Th229", "Th230", "Th231", "Th232", "Th233", "Th234", "Pa231", "Pa232", "Pa233", "U232", "U233", "U234", "U235", "U236", "U237", "U238", "U239", "U240", "U241", "Np235", "Np236", "Np237", "Np238", "Np239", "Pu236", "Pu237", "Pu238", "Pu239", "Pu240", "Pu241", "Pu242", "Pu243", "Pu244", "Am241", "Am242", "Am242m", "Am243", "Am244", "Am244m", "Cm240", "Cm241", "Cm242", "Cm243", "Cm244", "Cm245", "Cm246", "Cm247", "Cm248", "Cm249", "Cm250", "thorium", "protactinium", "uranium", "neptunium", "plutonium", "americium", "curium", "lost", "total"]

'["Th227", "Th228", "Th229", "Th230", "Th231", "Th232", "Th233", "Th234", "Pa231", "Pa232", "Pa233", "U232", "U233", "U234", "U235", "U236", "U237", "U238", "U239", "U240", "U241", "Np235", "Np236", "Np237", "Np238", "Np239", "Pu236", "Pu237", "Pu238", "Pu239", "Pu240", "Pu241", "Pu242", "Pu243", "Pu244", "Am241", "Am242", "Am242m", "Am243", "Am244", "Am244m", "Cm240", "Cm241", "Cm242", "Cm243", "Cm244", "Cm245", "Cm246", "Cm247", "Cm248", "Cm249", "Cm250", "thorium", "protactinium", "uranium", "neptunium", "plutonium", "americium", "curium", "lost", "total"]'

In [73]:
totalfuel.getValues('days', 'a', dayPoints, iso)

TypeError: 'int' object is not iterable